Antes de empezar con la explicación de esta entrega, debemos mencionar la corrección de dos errores de la entrega anterior. Hemos corregido el fallo de  aplicar stemming y lemmatización  tanto a TF-IDF como a embeddings. Ahora, solo se aplcia a TD-IDF. Además, adaptamos el Word2Vec para que no tuviese más de 30 epochs.

Por otro lado, hemos sido capaces de aplicar shallow learning a las tres tareas de clasificación que teníamos previstas. Sin embargo, solo hemos logrado implementar deep learning y la comparación de embeddings a la clasificación de sesgo. Las dos tareas restantes estarán completadas para la siguiente entrega.

# **1. Shallow Learning**

In [1]:
import os
import pickle
import warnings
import numpy as np
import pandas as pd
import torch
import tensorflow as tf
from collections import Counter
from gensim.models import Word2Vec, FastText
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC, SVC
from sentence_transformers import SentenceTransformer
from transformers import BertTokenizer, BertModel
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout, Embedding, Input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from xgboost import XGBClassifier


Para representar los textos, hemos elegido TF-IDF para capturar la importancia relativa de cada palabra en un documento frente al corpus completo, reduciendo el peso de palabras muy frecuentes que no aportan información discriminativa, como artículos y preposiciones en inglés. Hemos establecido un límite de 3,000 palabras más importantes para reducir la dimensionalidad. Además, hemos añadido unigramas y bigramas para capturar algo de contexto local sin sobrecargar el modelo. Por otro lado, hemos eliminado las palabras vacías en inglés para centrar el análisis en palabras significativas. Esta representación genera vectores dispersos que son ideales para los modelos clásicos que usamos.

Hemos seleccionado cuatro modelos para evaluar el desempeño: Logistic Regression, LinearSVC, Random Forest y XGBoost. Esta combinación nos permite cubrir tanto modelos lineales como no lineales y comparar rapidez, precisión y estabilidad.

Antes de entrenar, hemos convertido los textos a números mediante label encoding, y realizado una división de train/validation del 80/20 para medir el rendimiento real y evitar overfitting. Además, hemos filtrado las clases con menos de dos registros, ya que la validación estratificada requiere al menos dos ejemplos por clase. La función recibe como parámetro la variable objetivo. En este caso, recibe las variables "bias", "topic" y "source", que son nuestras variables a clasificar.

Finalmente, todos los modelos y el vectorizador TF-IDF han sido guardados para su reutilización. Este pipeline de Shallow Learning funciona como un baseline sólido que nos permite medir la mejora que aportan las representaciones densas y contextuales de texto que se utilizarán en las fases posteriores.

In [3]:
def shallow_pipeline(df, target_col):
    # Preparamos el texto
    if "text_joined" not in df.columns:
        df["text_joined"] = df["tokens"].apply(lambda x: " ".join(x))
    texts = df["text_joined"].astype(str).tolist()
    labels = df[target_col].tolist()

    # Filtramos las clases con menos de 2 registros
    counts = Counter(labels)
    valid_classes = [c for c, cnt in counts.items() if cnt > 1]
    mask = [lbl in valid_classes for lbl in labels]
    texts = [t for t, m in zip(texts, mask) if m]
    labels = [l for l, m in zip(labels, mask) if m]

    # Codificamos las etiquetas
    le = LabelEncoder()
    y = le.fit_transform(labels)

    # Hacemos el train/validation split
    X_train_text, X_val_text, y_train, y_val = train_test_split(
        texts, y, test_size=0.2, random_state=42, stratify=y
    )

    # Aplicamos TF-IDF
    vectorizer = TfidfVectorizer(max_features=3000, stop_words='english', ngram_range=(1,2))
    X_train = vectorizer.fit_transform(X_train_text)
    X_val = vectorizer.transform(X_val_text)

    # Definimos los modelos
    models = {
        "Logistic Regression": LogisticRegression(max_iter=1000, n_jobs=-1),
        "LinearSVC": LinearSVC(),
        "Random Forest": RandomForestClassifier(n_estimators=150, n_jobs=-1),
       # "XGBoost": XGBClassifier(n_estimators=75, eval_metric="mlogloss", tree_method="hist", n_jobs=-1)
    }

    results = {}

    # Entrenamos, evaluamos y guardamos los modelos
    os.makedirs("data/models", exist_ok=True)
    for name, model in models.items():
        print(f"Entrenando {name}...")
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        results[name] = {
            "Accuracy": accuracy_score(y_val, y_pred),
            "Macro-F1": f1_score(y_val, y_pred, average="macro")
        }
        # Guardamos el modelo
        pickle.dump(model, open(f"data/models/{name.replace(' ', '_').lower()}.pkl", "wb"))

    # Guardamos el vectorizador
    os.makedirs("data/features", exist_ok=True)
    pickle.dump(vectorizer, open("data/features/tfidf_vectorizer.pkl", "wb"))

    return results


Hemos tenido que incluir un filtro para eliminar las clases que tenían menos de dos registros antes de hacer el train_test_split. Esto se debe a que tuvimos un error al utilizar el parámetro stratify=y, que requiere al menos dos ejemplos por clase para poder crear correctamente los conjuntos de entrenamiento y validación de manera estratificada. Sin este filtro, cualquier clase con un único ejemplo provocaría que la ejecución se detuviera, como ocurría previamente con la variable source. Al aplicar este filtrado, nos aseguramos de que solo se utilicen clases con suficiente cantidad de datos, garantizando que la partición estratificada funcione y evitando que el pipeline falle durante el entrenamiento.

Los resultados se analizarán en la sección 4 de este noteebook.

In [4]:
df = pd.read_pickle("data/data_clean/train_tokenized.pkl")

In [5]:
# Llamamos a la función con bias
results_bias = shallow_pipeline(df, "bias")
print(pd.DataFrame(results_bias).T)



Entrenando Logistic Regression...


c:\Users\Usuario\anaconda3\envs\nlp\Lib\site-packages\sklearn\linear_model\_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


Entrenando LinearSVC...
Entrenando Random Forest...
                     Accuracy  Macro-F1
Logistic Regression  0.702109  0.700091
LinearSVC            0.698713  0.696551
Random Forest        0.693710  0.690578


In [6]:
# Llamamos a la función con topic
results_topic = shallow_pipeline(df, "topic")
print(pd.DataFrame(results_topic).T)


Entrenando Logistic Regression...


c:\Users\Usuario\anaconda3\envs\nlp\Lib\site-packages\sklearn\linear_model\_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


Entrenando LinearSVC...
Entrenando Random Forest...
                     Accuracy  Macro-F1
Logistic Regression  0.581129  0.338987
LinearSVC            0.589171  0.410156
Random Forest        0.520372  0.244504


In [7]:
# Llamamos a la función con source
results_source = shallow_pipeline(df, "source")
print(pd.DataFrame(results_source).T)

Entrenando Logistic Regression...


c:\Users\Usuario\anaconda3\envs\nlp\Lib\site-packages\sklearn\linear_model\_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


Entrenando LinearSVC...
Entrenando Random Forest...
                     Accuracy  Macro-F1
Logistic Regression  0.503401  0.103117
LinearSVC            0.559076  0.219572
Random Forest        0.499642  0.127022


# **2. Modelos Deep**


En la parte de Deep Learning, hemos optado por utilizar redes neuronales recurrentes, específicamente LSTM y GRU, debido a su capacidad para capturar dependencias secuenciales en el texto. A diferencia de los modelos de Shallow Learning, que tratan cada palabra o n-grama de manera independiente, las RNNs permiten que la red recuerde información contextual de palabras anteriores en la secuencia, lo cual es crucial para nuestras tareas de clasificación de texto, donde el significado puede depender del orden de las palabras.

Para la representación de los textos, hemos empleado embeddings densos, utilizando tres enfoques distintos con Word2Vec: congelado, fine-tune y desde cero. En el caso de los embeddings congelados, utilizamos un modelo preentrenado de Word2Vec y lo fijamos durante el entrenamiento de la red, de manera que solo la LSTM o GRU aprenda a combinar los vectores preexistentes. Esto permite evaluar cuánto conocimiento semántico ya capturado en Word2Vec puede ayudar a la tarea sin modificarlo. En el enfoque de fine-tune, los embeddings inicializados con Word2Vec se ajustan durante el entrenamiento, permitiendo que la red adapte los vectores a las particularidades del dataset específico. Finalmente, la opción de embeddings entrenados desde cero crea vectores aleatorios que se aprenden completamente durante el entrenamiento, lo que permite que la red descubra representaciones óptimas para la tarea, aunque requiere más datos y tiempo de entrenamiento.

Hemos elegido LSTM y GRU, ya que cumple nuestra necesidad de comparar dos variantes de redes recurrentes: las LSTM tienen una mayor capacidad para capturar dependencias de largo plazo mediante su mecanismo de puertas, mientras que las GRU son más simples y computacionalmente eficientes, lo que puede acelerar el entrenamiento sin perder demasiado rendimiento.

Los textos se transforman primero en secuencias de índices según el vocabulario de Word2Vec o un tokenizer entrenado sobre el dataset, y se aplica padding para unificar la longitud de las secuencias. Esto asegura que las redes puedan procesar lotes de datos de manera eficiente. Finalmente, la capa de salida utiliza softmax para producir probabilidades sobre las clases, y la red se entrena con categorical crossentropy, optimizando la accuracy y el macro-F1 como métricas de desempeño, lo cual es consistente con la evaluación utilizada en la parte de Shallow Learning.

En conclusión, este apartado permite que nuestra red aprenda tanto representaciones densas de palabras como patrones secuenciales de las oraciones, ofreciendo una ventaja sobre los modelos lineales y de ensamble de Shallow Learning que solo utilizan información superficial y dispersa de los textos.

Embeddings fine-tuneados

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec
import numpy as np
import pandas as pd

# ----------------------------
# 1. Cargar dataset y etiquetas
# ----------------------------
df_train = pd.read_pickle("data/data_clean/train_tokenized.pkl")
y = df_train["bias"].values

# Codificamos las labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_cat = to_categorical(y_encoded)

# ----------------------------
# 2. Train / Val / Test split
# ----------------------------
# Primero train + temp (val+test)
X_train_texts, X_temp_texts, y_train, y_temp = train_test_split(
    df_train["tokens"], y_cat, test_size=0.3, random_state=42, stratify=y_cat
)

# Luego temp → val + test
X_val_texts, X_test_texts, y_val, y_test = train_test_split(
    X_temp_texts, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

# ----------------------------
# 3. Word2Vec embeddings
# ----------------------------
w2v_model = Word2Vec.load("data/embeddings/word2vec.model")
embedding_dim = w2v_model.vector_size
word_index = {word: i+1 for i, word in enumerate(w2v_model.wv.index_to_key)}
vocab_size = len(word_index) + 1  # +1 para padding

def tokens_to_indices(tokens, word_index):
    return [word_index[t] for t in tokens if t in word_index]

X_train_idx = [tokens_to_indices(t, word_index) for t in X_train_texts]
X_val_idx = [tokens_to_indices(t, word_index) for t in X_val_texts]
X_test_idx = [tokens_to_indices(t, word_index) for t in X_test_texts]

max_seq_len = 200
X_train_pad = pad_sequences(X_train_idx, maxlen=max_seq_len, padding='post')
X_val_pad = pad_sequences(X_val_idx, maxlen=max_seq_len, padding='post')
X_test_pad = pad_sequences(X_test_idx, maxlen=max_seq_len, padding='post')

embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in word_index.items():
    embedding_matrix[i] = w2v_model.wv[word]

# ----------------------------
# 4. Definición del modelo RNN
# ----------------------------
def build_rnn(model_type='LSTM'):
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size,
                        output_dim=embedding_dim,
                        weights=[embedding_matrix],
                        input_length=max_seq_len,
                        trainable=True))  # Fine-tune embeddings
    if model_type == 'LSTM':
        model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
    elif model_type == 'GRU':
        model.add(GRU(128, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(y_cat.shape[1], activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=1e-3),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# ----------------------------
# 5. Early stopping
# ----------------------------
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# ----------------------------
# 6. Entrenamiento y evaluación
# ----------------------------
# LSTM
lstm_model = build_rnn('LSTM')
lstm_history = lstm_model.fit(
    X_train_pad, y_train,
    validation_data=(X_val_pad, y_val),
    epochs=20,
    batch_size=64,
    callbacks=[early_stop]
)

# GRU
gru_model = build_rnn('GRU')
gru_history = gru_model.fit(
    X_train_pad, y_train,
    validation_data=(X_val_pad, y_val),
    epochs=20,
    batch_size=64,
    callbacks=[early_stop]
)

# ----------------------------
# 7. Predicciones sobre TEST
# ----------------------------
y_pred_lstm = np.argmax(lstm_model.predict(X_test_pad, batch_size=64), axis=1)
y_pred_gru = np.argmax(gru_model.predict(X_test_pad, batch_size=64), axis=1)
y_test_labels = np.argmax(y_test, axis=1)

# ----------------------------
# 8. Métricas finales
# ----------------------------
results = {
    'LSTM': {
        'Accuracy': accuracy_score(y_test_labels, y_pred_lstm),
        'Macro-F1': f1_score(y_test_labels, y_pred_lstm, average='macro')
    },
    'GRU': {
        'Accuracy': accuracy_score(y_test_labels, y_pred_gru),
        'Macro-F1': f1_score(y_test_labels, y_pred_gru, average='macro')
    }
}

results_df = pd.DataFrame(results).T
print("Resultados finales sobre TEST:")
print(results_df)


c:\Users\Usuario\anaconda3\envs\nlp\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/20
306/306 ━━━━━━━━━━━━━━━━━━━━ 182s 567ms/step - accuracy: 0.4150 - loss: 1.0662 - val_accuracy: 0.4610 - val_loss: 1.0382
Epoch 2/20
306/306 ━━━━━━━━━━━━━━━━━━━━ 200s 563ms/step - accuracy: 0.5123 - loss: 0.9728 - val_accuracy: 0.4803 - val_loss: 1.0051
Epoch 3/20
306/306 ━━━━━━━━━━━━━━━━━━━━ 187s 610ms/step - accuracy: 0.5791 - loss: 0.8858 - val_accuracy: 0.5013 - val_loss: 0.9718
Epoch 4/20
306/306 ━━━━━━━━━━━━━━━━━━━━ 140s 458ms/step - accuracy: 0.6515 - loss: 0.7710 - val_accuracy: 0.5127 - val_loss: 1.0037
Epoch 5/20
306/306 ━━━━━━━━━━━━━━━━━━━━ 174s 564ms/step - accuracy: 0.7315 - loss: 0.6269 - val_accuracy: 0.5096 - val_loss: 1.1131
Epoch 6/20
306/306 ━━━━━━━━━━━━━━━━━━━━ 156s 512ms/step - accuracy: 0.8044 - loss: 0.4847 - val_accuracy: 0.5037 - val_loss: 1.2262
Epoch 1/20
306/306 ━━━━━━━━━━━━━━━━━━━━ 124s 393ms/step - accuracy: 0.3974 - loss: 1.0817 - val_accuracy: 0.4224 - val_loss: 1.0602
Epoch 2/20
306/306 ━━━━━━━━━━━━━━━━━━━━ 152s 425ms/step - accuracy: 0.4762 -

Embeddings no finetuneados

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, f1_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import pandas as pd

# Creamos la columna text_joined a partir de tokens
df["text_joined"] = df["tokens"].apply(lambda x: " ".join(x))

texts = df["text_joined"].astype(str).tolist()
labels = df["bias"].tolist()

# Tokenización y secuencias
vocab_size = 20000
maxlen = 100
embedding_dim = 100

tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(texts)
X_seq = tokenizer.texts_to_sequences(texts)
X = pad_sequences(X_seq, maxlen=maxlen)

# Codificamos las etiquetas
le = LabelEncoder()
y = le.fit_transform(labels)

# Train / Validation / Test split (60/20/20)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# Funciones para LSTM y GRU
def build_lstm_model():
    model = Sequential([
        Embedding(vocab_size, embedding_dim, input_length=maxlen, trainable=False),
        LSTM(128, return_sequences=False),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(len(np.unique(y)), activation='softmax')
    ])
    model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

def build_gru_model():
    model = Sequential([
        Embedding(vocab_size, embedding_dim, input_length=maxlen, trainable=False),
        GRU(128, return_sequences=False),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(len(np.unique(y)), activation='softmax')
    ])
    model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Entrenamiento
lstm_model = build_lstm_model()
history_lstm = lstm_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=64,
    callbacks=[early_stopping]
)

gru_model = build_gru_model()
history_gru = gru_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=64,
    callbacks=[early_stopping]
)

# Evaluación por clase
def evaluate(model, X_test, y_test):
    preds = np.argmax(model.predict(X_test, batch_size=64), axis=1)
    print(classification_report(y_test, preds, target_names=le.classes_))
    acc = accuracy_score(y_test, preds)
    f1 = f1_score(y_test, preds, average='macro')
    return acc, f1

print("\nResultados Embedding Random (NO fine-tuneado)")
acc_lstm, f1_lstm = evaluate(lstm_model, X_test, y_test)
acc_gru, f1_gru = evaluate(gru_model, X_test, y_test)

# Guardamos resultados en DataFrame
results_df = pd.DataFrame({
    'Model': ['LSTM', 'GRU'],
    'Accuracy': [acc_lstm, acc_gru],
    'Macro-F1': [f1_lstm, f1_gru]
})
print(results_df)


NameError: name 'df' is not defined

Word2Vec congelado vs Word2Vec fine-tuneado vs Word2Vec from scratch

In [ ]:
from gensim.models import Word2Vec

# Cargamos el dataset tokenizado
df = pd.read_pickle("data/data_clean/train_tokenized.pkl")
df["text_joined"] = df["tokens"].apply(lambda x: " ".join(x))

texts = df["tokens"].tolist()
labels = df["bias"].tolist()

# Codificamos las etiquetas
le = LabelEncoder()
y = le.fit_transform(labels)

# Train / Validation / Test split (60/20/20)
X_train_text, X_temp_text, y_train, y_temp = train_test_split(texts, y, test_size=0.4, random_state=42, stratify=y)
X_val_text, X_test_text, y_val, y_test = train_test_split(X_temp_text, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# Cargamos el Word2Vec preentrenado
w2v_model = Word2Vec.load("data/embeddings/word2vec.model")
embedding_dim = w2v_model.vector_size

word_index = {word: i+1 for i, word in enumerate(w2v_model.wv.index_to_key)}
vocab_size = len(word_index) + 1

def tokens_to_indices(tokens, word_index):
    return [word_index[t] for t in tokens if t in word_index]

X_train_idx = [tokens_to_indices(t, word_index) for t in X_train_text]
X_val_idx = [tokens_to_indices(t, word_index) for t in X_val_text]
X_test_idx = [tokens_to_indices(t, word_index) for t in X_test_text]

max_seq_len = 200
X_train_pad = pad_sequences(X_train_idx, maxlen=max_seq_len, padding='post')
X_val_pad = pad_sequences(X_val_idx, maxlen=max_seq_len, padding='post')
X_test_pad = pad_sequences(X_test_idx, maxlen=max_seq_len, padding='post')

embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in word_index.items():
    embedding_matrix[i] = w2v_model.wv[word]

# Función para LSTM con embeddings preentrenados
from tensorflow.keras.optimizers import Adam
def build_lstm_model(embedding_matrix, trainable=True):
    model = Sequential()
    model.add(Embedding(
        input_dim=embedding_matrix.shape[0],
        output_dim=embedding_matrix.shape[1],
        weights=[embedding_matrix],
        input_length=max_seq_len,
        trainable=trainable
    ))
    model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer=Adam(1e-3), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Word2Vec Frozen
lstm_frozen = build_lstm_model(embedding_matrix, trainable=False)
history_frozen = lstm_frozen.fit(
    X_train_pad, y_train,
    validation_data=(X_val_pad, y_val),
    epochs=20,
    batch_size=64,
    callbacks=[early_stopping]
)

# Word2Vec Fine-tune
lstm_finetune = build_lstm_model(embedding_matrix, trainable=True)
history_finetune = lstm_finetune.fit(
    X_train_pad, y_train,
    validation_data=(X_val_pad, y_val),
    epochs=20,
    batch_size=64,
    callbacks=[early_stopping]
)

# Word2Vec Scratch
embedding_matrix_random = np.random.normal(size=(vocab_size, embedding_dim))
lstm_scratch = build_lstm_model(embedding_matrix_random, trainable=True)
history_scratch = lstm_scratch.fit(
    X_train_pad, y_train,
    validation_data=(X_val_pad, y_val),
    epochs=20,
    batch_size=64,
    callbacks=[early_stopping]
)

# Evaluación por clase
def evaluate(model, X_test, y_test):
    preds = np.argmax(model.predict(X_test, batch_size=64), axis=1)
    print(classification_report(y_test, preds, target_names=le.classes_))
    acc = accuracy_score(y_test, preds)
    f1 = f1_score(y_test, preds, average='macro')
    return acc, f1

results = {}
print("\nResultados Word2Vec Frozen")
results['Word2Vec Frozen'] = evaluate(lstm_frozen, X_test_pad, y_test)

print("\nResultados Word2Vec Fine-tune")
results['Word2Vec Fine-tune'] = evaluate(lstm_finetune, X_test_pad, y_test)

print("\nResultados Word2Vec Scratch")
results['Word2Vec Scratch'] = evaluate(lstm_scratch, X_test_pad, y_test)

results_df = pd.DataFrame(results, index=['Accuracy','Macro-F1']).T
print(results_df)


c:\Users\Usuario\anaconda3\envs\nlp\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
350/350 ━━━━━━━━━━━━━━━━━━━━ 268s 737ms/step - accuracy: 0.4120 - loss: 1.0719 - val_accuracy: 0.4355 - val_loss: 1.0549
Epoch 2/5
350/350 ━━━━━━━━━━━━━━━━━━━━ 357s 1s/step - accuracy: 0.4564 - loss: 1.0241 - val_accuracy: 0.4694 - val_loss: 1.0021
Epoch 3/5
350/350 ━━━━━━━━━━━━━━━━━━━━ 201s 575ms/step - accuracy: 0.4739 - loss: 0.9988 - val_accuracy: 0.4682 - val_loss: 0.9979
Epoch 4/5
350/350 ━━━━━━━━━━━━━━━━━━━━ 201s 575ms/step - accuracy: 0.4852 - loss: 0.9866 - val_accuracy: 0.4973 - val_loss: 0.9732
Epoch 5/5
350/350 ━━━━━━━━━━━━━━━━━━━━ 152s 433ms/step - accuracy: 0.4917 - loss: 0.9776 - val_accuracy: 0.4993 - val_loss: 0.9686
Epoch 1/5
350/350 ━━━━━━━━━━━━━━━━━━━━ 143s 398ms/step - accuracy: 0.4306 - loss: 1.0563 - val_accuracy: 0.4725 - val_loss: 1.0097
Epoch 2/5
350/350 ━━━━━━━━━━━━━━━━━━━━ 181s 510ms/step - accuracy: 0.5197 - loss: 0.9549 - val_accuracy: 0.4961 - val_loss: 0.9732
Epoch 3/5
350/350 ━━━━━━━━━━━━━━━━━━━━ 146s 349ms/step - accuracy: 0.5951 - loss: 0.85

# **3. Comparación de Embeddings**


En esta sección se consideran distintas técnicas de representación de texto para tareas de clasificación orientadas a la detección de sesgos ideológicos, tema y medio en artículos periodísticos. Para ello se analizan enfoques tradicionales, embeddings no contextuales y embeddings contextuales, con el objetivo de entender cómo cada representación captura información relevante del lenguaje dentro de este dominio específico.

Los métodos tradicionales, como TF-IDF y Bag-of-Words (BoW), representan el texto mediante vectores dispersos basados únicamente en la frecuencia o presencia de términos, sin tener en cuenta el word order ni el context. Se espera que estos enfoques funcionen bien cuando ciertas palabras clave o expresiones son indicadores directos de postura ideológica o del tema tratado. Sin embargo, su capacidad para capturar matices ideológicos sutiles, estructuras discursivas o patrones retóricos es limitada debido a la ausencia de información contextual.

Los embeddings no contextuales, como Word2Vec y FastText, generan dense word vectors aprendidos a partir de coocurrencias en un corpus. Estos modelos son capaces de capturar similitudes semánticas entre palabras y asociaciones típicas del lenguaje periodístico, lo que ayuda a identificar vocabulario característico de ciertos medios o tendencias ideológicas. Aunque estos embeddings proporcionan una representación más rica que los métodos tradicionales, no distinguen los diferentes significados de una palabra según el contexto en el que aparece. En el caso de FastText, el uso de subword embeddings permite manejar mejor palabras raras, neologismos o términos específicos de determinados medios.

Finalmente, los embeddings contextuales, como Sentence Transformers o BERT, generan representaciones que dependen del contexto completo de la oración o del documento. Esto permite que una misma palabra tenga diferentes vectores según su significado en el artículo, capturando relaciones semánticas complejas, long-range dependencies y matices ideológicos implícitos. Se espera que estos modelos sean especialmente efectivos para detectar bias más sutil, diferencias discursivas entre medios y patrones retóricos que dependen del estilo o la narrativa del artículo. No obstante, este tipo de modelos suele requerir un mayor volumen de datos y mayor capacidad computacional para ajustarse correctamente a tareas especializadas como la clasificación ideológica o la identificación del medio.



**3.1 Embeddings tradicionales**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import pandas as pd

# Cargamos y preparamos los datos
df_train = pd.read_pickle("data/data_clean/train_tokenized.pkl")
df_train["text_joined"] = df_train["tokens"].apply(lambda x: " ".join(x))
texts = df_train["text_joined"].tolist()
y = df_train["bias"].values

results = {}

# -------------------
# Train / Val / Test
# -------------------
X_temp, X_test, y_temp, y_test = train_test_split(texts, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp)
# Esto da 60% train / 20% val / 20% test

# -------------------
# TF-IDF
# -------------------
tfidf = TfidfVectorizer(max_features=5000, stop_words='english', ngram_range=(1,2))
X_train_tfidf = tfidf.fit_transform(X_train)
X_val_tfidf = tfidf.transform(X_val)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_tfidf, y_train)

y_val_pred = clf.predict(X_val_tfidf)
y_test_pred = clf.predict(X_test_tfidf)

results["TF-IDF"] = {
    "Val Accuracy": accuracy_score(y_val, y_val_pred),
    "Test Accuracy": accuracy_score(y_test, y_test_pred),
    "Val F1 (weighted)": f1_score(y_val, y_val_pred, average="weighted"),
    "Test F1 (weighted)": f1_score(y_test, y_test_pred, average="weighted")
}

print("TF-IDF - Classification Report (Test):")
print(classification_report(y_test, y_test_pred))

# -------------------
# Bag-of-Words
# -------------------
bow = CountVectorizer(max_features=5000, stop_words='english', ngram_range=(1,2))
X_train_bow = bow.fit_transform(X_train)
X_val_bow = bow.transform(X_val)
X_test_bow = bow.transform(X_test)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_bow, y_train)

y_val_pred = clf.predict(X_val_bow)
y_test_pred = clf.predict(X_test_bow)

results["Bag-of-Words"] = {
    "Val Accuracy": accuracy_score(y_val, y_val_pred),
    "Test Accuracy": accuracy_score(y_test, y_test_pred),
    "Val F1 (weighted)": f1_score(y_val, y_val_pred, average="weighted"),
    "Test F1 (weighted)": f1_score(y_test, y_test_pred, average="weighted")
}

print("Bag-of-Words - Classification Report (Test):")
print(classification_report(y_test, y_test_pred))

# -------------------
# Resultados
# -------------------
results_df = pd.DataFrame(results).T
print("\nComparativa Embeddings Tradicionales:")
print(results_df)


,Accuracy,F1-score
TF-IDF,0.703002,0.702330
Bag-of-Words,0.646712,0.646754


**3.2 Embeddings no contextuales**

In [ ]:
from gensim.models import Word2Vec, FastText
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
import numpy as np

# Preparar tokens
df_train = pd.read_pickle("data/data_clean/train_tokenized.pkl")
sentences = df_train["tokens"].tolist()
y = df_train["bias"].values

# Train / Val / Test split (60/20/20)
X_temp, X_test, y_temp, y_test = train_test_split(sentences, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp)

results = {}

# -------------------
# Word2Vec
# -------------------
w2v_model = Word2Vec(sentences=X_train, vector_size=100, window=5, min_count=3, workers=1, sg=1)
w2v_model.save("data/embeddings/word2vec.model")

# Promediar embeddings por frase
def get_avg_w2v(sentence, model):
    vecs = [model.wv[word] for word in sentence if word in model.wv]
    if len(vecs) == 0:
        return np.zeros(model.vector_size)
    return np.mean(vecs, axis=0)

X_train_vec = np.array([get_avg_w2v(s, w2v_model) for s in X_train])
X_val_vec = np.array([get_avg_w2v(s, w2v_model) for s in X_val])
X_test_vec = np.array([get_avg_w2v(s, w2v_model) for s in X_test])

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_vec, y_train)
y_val_pred = clf.predict(X_val_vec)
y_test_pred = clf.predict(X_test_vec)

results["Word2Vec"] = {
    "Val Accuracy": accuracy_score(y_val, y_val_pred),
    "Test Accuracy": accuracy_score(y_test, y_test_pred),
    "Val F1 (weighted)": f1_score(y_val, y_val_pred, average="weighted"),
    "Test F1 (weighted)": f1_score(y_test, y_test_pred, average="weighted")
}

print("Word2Vec - Classification Report (Test):")
print(classification_report(y_test, y_test_pred))

# -------------------
# FastText
# -------------------
fasttext_model = FastText(sentences=X_train, vector_size=100, window=5, min_count=3, workers=1, sg=1)
fasttext_model.save("data/embeddings/fasttext.model")

X_train_vec = np.array([get_avg_w2v(s, fasttext_model) for s in X_train])
X_val_vec = np.array([get_avg_w2v(s, fasttext_model) for s in X_val])
X_test_vec = np.array([get_avg_w2v(s, fasttext_model) for s in X_test])

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_vec, y_train)
y_val_pred = clf.predict(X_val_vec)
y_test_pred = clf.predict(X_test_vec)

results["FastText"] = {
    "Val Accuracy": accuracy_score(y_val, y_val_pred),
    "Test Accuracy": accuracy_score(y_test, y_test_pred),
    "Val F1 (weighted)": f1_score(y_val, y_val_pred, average="weighted"),
    "Test F1 (weighted)": f1_score(y_test, y_test_pred, average="weighted")
}

print("FastText - Classification Report (Test):")
print(classification_report(y_test, y_test_pred))

# -------------------
# Resultados finales
# -------------------
results_df = pd.DataFrame(results).T
print("\nComparativa Embeddings No Contextuales:")
print(results_df)


Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_fl

Downstream (clustering) Word2Vec: 184242.40625


Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_fl

Downstream (clustering) FastText: 258313.15625


**3.3 Embeddings contextuales**

In [ ]:
from sentence_transformers import SentenceTransformer
from transformers import BertTokenizer, BertModel
import torch
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

# Preparar textos
df_train = pd.read_pickle("data/data_clean/train_tokenized.pkl")
df_train["text_joined"] = df_train["tokens"].apply(lambda x: " ".join(x))
texts = df_train["text_joined"].tolist()
y = df_train["bias"].values

# Train / Val / Test split
X_temp, X_test, y_temp, y_test = train_test_split(texts, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp)

results = {}

# -------------------
# Sentence Transformers
# -------------------
st_model = SentenceTransformer("all-MiniLM-L6-v2")
X_train_vec = st_model.encode(X_train, batch_size=32, show_progress_bar=True)
X_val_vec = st_model.encode(X_val, batch_size=32)
X_test_vec = st_model.encode(X_test, batch_size=32)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_vec, y_train)
y_val_pred = clf.predict(X_val_vec)
y_test_pred = clf.predict(X_test_vec)

results["Sentence Transformers"] = {
    "Val Accuracy": accuracy_score(y_val, y_val_pred),
    "Test Accuracy": accuracy_score(y_test, y_test_pred),
    "Val F1 (weighted)": f1_score(y_val, y_val_pred, average="weighted"),
    "Test F1 (weighted)": f1_score(y_test, y_test_pred, average="weighted")
}

print("Sentence Transformers - Classification Report (Test):")
print(classification_report(y_test, y_test_pred))

# -------------------
# BERT (limitado a 100 textos para no saturar memoria)
# -------------------
bert_model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
bert_model = BertModel.from_pretrained(bert_model_name)
bert_model.eval()

def bert_sentence_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = bert_model(**inputs)
        embeddings = outputs.last_hidden_state.squeeze(0)
        return embeddings.mean(dim=0).numpy()

X_train_subset = X_train[:100]
X_val_subset = X_val[:20]
X_test_subset = X_test[:20]
y_train_subset = y_train[:100]
y_val_subset = y_val[:20]
y_test_subset = y_test[:20]

X_train_vec = np.array([bert_sentence_embedding(t) for t in X_train_subset])
X_val_vec = np.array([bert_sentence_embedding(t) for t in X_val_subset])
X_test_vec = np.array([bert_sentence_embedding(t) for t in X_test_subset])

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_vec, y_train_subset)
y_val_pred = clf.predict(X_val_vec)
y_test_pred = clf.predict(X_test_vec)

results["BERT"] = {
    "Val Accuracy": accuracy_score(y_val_subset, y_val_pred),
    "Test Accuracy": accuracy_score(y_test_subset, y_test_pred),
    "Val F1 (weighted)": f1_score(y_val_subset, y_val_pred, average="weighted"),
    "Test F1 (weighted)": f1_score(y_test_subset, y_test_pred, average="weighted")
}

print("BERT - Classification Report (Test):")
print(classification_report(y_test_subset, y_test_pred))

# -------------------
# Resultados finales
# -------------------
results_df = pd.DataFrame(results).T
print("\nComparativa Embeddings Contextuales:")
print(results_df)


Batches:   0%|          | 0/875 [00:00<?, ?it/s]

Sentence Transformers - Accuracy: 0.5394924946390279
Sentence Transformers - F1 Score: 0.5379302986979321
BERT - Accuracy: 0.3
BERT - F1 Score: 0.25047619047619046


# **4. Tabla Comparativa de Resultados**

**4.1 Shallow Learning**

Para entender los resultados, hay que aclarar los parámetros utilizados para cada variable en Shallow Learning.

En el caso de la variable source, hemos tenido que reducir el número de estimadores del Random Forest de 300 a 150. Además, el número de estimadores del XGBoost también ha sido reducido de 200 a 75. Sin esta reducción, no habríamos sido capaces de terminar la ejecución de la celda. Ha llegado a estar más de una hora y seguía sin terminar de ejecutarse.

En cuanto a la variable topic, además de las rebajas aplicadas al caso de la variable source, hemos decidido quitar el modelo XGBoost, ya que no termina de ejecutarse. Disponemos de equipos con capacidades técnicas muy limitadas, por lo que, con mejores ordenadores, no se tendrían que reducir los valores.

In [ ]:
# Resultados de clasificación por variable objetivo
results_bias = {
    "Logistic Regression": {"Accuracy": 0.702109, "Macro-F1": 0.700091},
    "LinearSVC": {"Accuracy": 0.698713, "Macro-F1": 0.696551},
    "Random Forest": {"Accuracy": 0.683881, "Macro-F1": 0.679829},
    "XGBoost": {"Accuracy": 0.734632, "Macro-F1": 0.733918}
}

results_topic = {
    "Logistic Regression": {"Accuracy": 0.581129, "Macro-F1": 0.338987},
    "LinearSVC": {"Accuracy": 0.589171, "Macro-F1": 0.410156},
    "Random Forest": {"Accuracy": 0.519657, "Macro-F1": 0.250003},
    "XGBoost": {"Accuracy": 0.0, "Macro-F1": 0.0}
}

results_source = {
    "Logistic Regression": {"Accuracy": 0.503401, "Macro-F1": 0.103117},
    "LinearSVC": {"Accuracy": 0.559076, "Macro-F1": 0.219572},
    "Random Forest": {"Accuracy": 0.500000, "Macro-F1": 0.120979},
    "XGBoost": {"Accuracy": 0.532581, "Macro-F1": 0.223251}
}

# --- Creación del DataFrame de Comparación ---
rows = []
models = ["Logistic Regression", "LinearSVC", "Random Forest", "XGBoost"]

for model in models:
    row = {
        "Model": model,
        "Bias Accuracy": results_bias[model]["Accuracy"],
        "Bias Macro-F1": results_bias[model]["Macro-F1"],
        "Topic Accuracy": results_topic[model]["Accuracy"],
        "Topic Macro-F1": results_topic[model]["Macro-F1"],
        "Source Accuracy": results_source[model]["Accuracy"],
        "Source Macro-F1": results_source[model]["Macro-F1"]
    }
    rows.append(row)

df_comparison = pd.DataFrame(rows)

# --- Formateo para la presentación ---

# Redondear todas las columnas de métricas a 4 decimales
df_display = df_comparison.round(4)

# Crear un multi-índice para los nombres de las columnas para agrupar las métricas
cols = [('Métricas', 'Model'), 
        ('Bias', 'Accuracy'), ('Bias', 'Macro-F1'),
        ('Topic', 'Accuracy'), ('Topic', 'Macro-F1'),
        ('Source', 'Accuracy'), ('Source', 'Macro-F1')]

df_display.columns = pd.MultiIndex.from_tuples(cols)

# Imprimir la tabla
print(" Resultados de Clasificación Shallow Learning (TF-IDF)\n")
# Usar to_markdown o to_string para una salida limpia en consola
print(df_display.to_string(index=False))



 Resultados de Clasificación Shallow Learning (TF-IDF)

           Métricas     Bias             Topic            Source         
              Model Accuracy Macro-F1 Accuracy Macro-F1 Accuracy Macro-F1
Logistic Regression   0.7021   0.7001   0.5811   0.3390   0.5034   0.1031
          LinearSVC   0.6987   0.6966   0.5892   0.4102   0.5591   0.2196
      Random Forest   0.6839   0.6798   0.5197   0.2500   0.5000   0.1210
            XGBoost   0.7346   0.7339   0.0000   0.0000   0.5326   0.2233


Los resultados de clasificación usando Shallow Learning con TF-IDF muestran un comportamiento variable según la tarea y el modelo. Para la tarea de Bias, todos los modelos presentan un desempeño sólido, con XGBoost liderando (accuracy 0.7346, macro-F1 0.7339) seguido de Logistic Regression y LinearSVC alrededor de 0.70 en ambas métricas. Esto indica que las diferencias de sesgo en los textos son relativamente fáciles de capturar con TF-IDF.

En la tarea de Topic, los modelos lineales, especialmente LinearSVC, obtienen los mejores resultados (accuracy 0.5892, macro-F1 0.4102). La celda de XGBoost muestra 0, al no haberse aplicado este algoritmo a la tarea por los problemas de tiempo de ejecucion explicados anteriormente. Esto refuerza que, para la clasificación por tópicos, los modelos lineales son adecuados con TF-IDF, que captura bien los términos distintivos de cada tema.

En la tarea de Source, el desempeño general es más bajo. LinearSVC alcanza un accuracy de 0.5591 y un macro-F1 de 0.2196, mientras que Logistic Regression y otros modelos muestran valores menores. Esto refleja la dificultad de diferenciar la fuente del texto únicamente con TF-IDF, ya que las diferencias estilísticas son menos evidentes en la representación basada en frecuencias de palabras.

En resumen, los modelos lineales ofrecen resultados consistentes para Bias y Topic, mientras que los modelos de ensamble como Random Forest o XGBoost se aplican solo a algunas tareas y muestran fortalezas específicas. TF-IDF es útil para capturar patrones globales de sesgo y tópico, pero es limitado para distinguir la fuente de los textos.

**4.2 Deep Learning**

In [ ]:
# Datos proporcionados
data_finetune = {
    "Modelo": ["LSTM", "GRU"],
    "Estrategia de Embedding": ["Finetuneado (Aprende)", "Finetuneado (Aprende)"],
    "Accuracy": [0.513402, 0.526447],
    "Macro-F1": [0.514574, 0.526548]
}

data_random = {
    "Modelo": ["LSTM", "GRU"],
    "Estrategia de Embedding": ["No Finetuneado (Random)", "No Finetuneado (Random)"],
    "Accuracy": [0.39867762687634023, 0.3858112937812723],
    "Macro-F1": [0.33385069350207125, 0.3545762899287412]
}

data_word2vec = {
    "Modelo": ["Word2Vec", "Word2Vec", "Word2Vec"], # Usamos Word2Vec como modelo en este caso para distinguirlo
    "Estrategia de Embedding": ["Word2Vec (Frozen)", "Word2Vec (Fine-tune)", "Word2Vec (Scratch)"],
    "Accuracy": [0.499285, 0.502323, 0.492137],
    "Macro-F1": [0.487054, 0.502254, 0.487125]
}

# Crear DataFrames
df_finetune = pd.DataFrame(data_finetune)
df_random = pd.DataFrame(data_random)
df_word2vec = pd.DataFrame(data_word2vec)

# Unir todos los DataFrames
df_deep_learning = pd.concat([df_finetune, df_random, df_word2vec], ignore_index=True)

# Redondear las métricas a 4 decimales para la presentación
df_deep_learning_display = df_deep_learning.round(4)

# Reordenar las columnas
column_order = ["Estrategia de Embedding", "Modelo", "Accuracy", "Macro-F1"]
df_deep_learning_display = df_deep_learning_display[column_order]

# Imprimir la tabla
print(" Rendimiento de Modelos de Deep Learning (LSTM & GRU)\n")
# Usar to_string para una salida de tabla limpia en consola
print(df_deep_learning_display.to_string(index=False))

 Rendimiento de Modelos de Deep Learning (LSTM & GRU)

Estrategia de Embedding   Modelo  Accuracy  Macro-F1
  Finetuneado (Aprende)     LSTM    0.5134    0.5146
  Finetuneado (Aprende)      GRU    0.5264    0.5265
No Finetuneado (Random)     LSTM    0.3987    0.3339
No Finetuneado (Random)      GRU    0.3858    0.3546
      Word2Vec (Frozen) Word2Vec    0.4993    0.4871
   Word2Vec (Fine-tune) Word2Vec    0.5023    0.5023
     Word2Vec (Scratch) Word2Vec    0.4921    0.4871


Los modelos de Deep Learning muestran un comportamiento muy dependiente de la estrategia de embeddings utilizada y del tipo de arquitectura. En primer lugar, los modelos finetuneados, que ajustan los embeddings durante el entrenamiento, obtienen el mejor desempeño general. Tanto LSTM como GRU alcanzan accuracy y macro-F1 superiores a 0.51, con GRU ligeramente por encima de LSTM (0.5264 vs 0.5134 en accuracy). Esto sugiere que permitir que los embeddings aprendan junto con el modelo proporciona representaciones más ajustadas a la tarea y mejora la capacidad de generalización.

En contraste, los modelos no finetuneados con embeddings aleatorios presentan un desempeño mucho más bajo, con accuracy alrededor de 0.39-0.40 y macro-F1 entre 0.33 y 0.35. Esto refleja que sin preentrenamiento ni ajuste, los modelos luchan por aprender representaciones significativas desde cero a partir de datos limitados. La ligera diferencia entre LSTM y GRU indica que la arquitectura por sí sola no es suficiente para compensar embeddings iniciales pobres.

Cuando se usan embeddings preentrenados tipo Word2Vec, el desempeño se estabiliza entre 0.49 y 0.50 en accuracy. Mantener los embeddings congelados ofrece resultados razonables (0.4993 accuracy), mientras que permitir un finetune ligero mejora apenas a 0.5023, lo que indica que la adaptación incremental no aporta grandes ganancias en este caso. Entrenar embeddings desde cero (scratch) también genera resultados similares (0.4921 accuracy), mostrando que el preentrenamiento ayuda, pero no de manera dramática, frente a una arquitectura LSTM o GRU básica.

**4.3 Embeddings**

In [ ]:
# Datos de los resultados de embeddings contextuales
results_contextual = {
    "Modelo/Técnica": ["Sentence Transformers", "BERT"],
    "Accuracy": [0.5394924946390279, 0.3],
    "F1 Score": [0.5379302986979321, 0.25047619047619046]
}

# Datos de los resultados de embeddings tradicionales (Shallow Learning)
results_traditional = {
    "Modelo/Técnica": ["TF-IDF", "Bag-of-Words (BoW)"],
    "Accuracy": [0.703002, 0.646712],
    "F1 Score": [0.702330, 0.646754]
}

# Datos de los resultados de embeddings no contextuales (Clustering Downstream)
# Nota: Estas métricas son diferentes (ej. Inercia/Distorsión)
results_clustering = {
    "Modelo/Técnica": ["Word2Vec", "FastText"],
    "Clustering Score (Downstream)": [184242.40625, 258313.15625]
}

# Crear DataFrames
df_contextual = pd.DataFrame(results_contextual).assign(Tipo_Embedding="Contextual")
df_traditional = pd.DataFrame(results_traditional).assign(Tipo_Embedding="Tradicional")
df_clustering = pd.DataFrame(results_clustering).assign(Tipo_Embedding="No Contextual ")

# Consolidar todos los DataFrames
# Usamos pd.concat y rellenamos las columnas faltantes automáticamente con NaN
df_results = pd.concat([df_contextual, df_traditional, df_clustering], ignore_index=True)

# --- Formateo y limpieza ---

# Redondear las métricas de clasificación a 4 decimales
df_results["Accuracy"] = df_results["Accuracy"].round(4)
df_results["F1 Score"] = df_results["F1 Score"].round(4)

# Formatear el Clustering Score a 2 decimales para claridad y usar separador de miles
df_results["Clustering Score (Downstream)"] = df_results["Clustering Score (Downstream)"].apply(
    lambda x: f"{x:,.2f}" if pd.notna(x) else np.nan
)

# Reordenar las columnas para una mejor presentación
column_order = ["Tipo_Embedding", "Modelo/Técnica", "Accuracy", "F1 Score", "Clustering Score (Downstream)"]
df_results = df_results[column_order]

# Renombrar la columna principal para la impresión
df_results.rename(columns={"Tipo_Embedding": "Tipo de Embedding"}, inplace=True)

# Imprimir la tabla
print("Resultados Consolidados de Modelos de Representación de Texto\n")
print(df_results.to_string(index=False))

Resultados Consolidados de Modelos de Representación de Texto

Tipo de Embedding        Modelo/Técnica  Accuracy  F1 Score Clustering Score (Downstream)
       Contextual Sentence Transformers    0.5395    0.5379                           NaN
       Contextual                  BERT    0.3000    0.2505                           NaN
      Tradicional                TF-IDF    0.7030    0.7023                           NaN
      Tradicional    Bag-of-Words (BoW)    0.6467    0.6468                           NaN
   No Contextual               Word2Vec       NaN       NaN                    184,242.41
   No Contextual               FastText       NaN       NaN                    258,313.16


Los resultados muestran diferencias claras según el tipo de embedding utilizado. Los embeddings tradicionales (TF-IDF y Bag-of-Words) lograron el mejor desempeño en la tarea de clasificación de la variable bias, con accuracy de 0.7030 y 0.6467, y F1 scores de 0.7023 y 0.6468 respectivamente, lo que indica que estos enfoques aún son muy efectivos para tareas de machine learning sobre texto. En contraste, los embeddings contextuales (Sentence Transformers y BERT) obtuvieron valores más bajos en Accuracy y F1, siendo Sentence Transformers ligeramente superior a BERT (0.5395 vs 0.3000 en Accuracy), aunque ambos generaron vectores de documento útiles. Por último, los embeddings no-contextuales (Word2Vec y FastText) no se evaluaron directamente en clasificación, pero su desempeño en clustering muestra que Word2Vec produjo clusters más compactos (184,242) comparado con FastText (258,313), reflejando que sus representaciones capturan cierta estructura semántica, aunque no directamente utilizable para clasificación sin agregación de vectores por documento.